In [120]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
import pandas as pd
import numpy as np
import pickle
import requests
import re

In [30]:
teams = {'Atlanta Falcons':'atl','Buffalo Bills':'buf','Carolina Panthers':'car','Chicago Bears':'chi',
         'Cincinnati Bengals':'cin','Cleveland Browns':'cle','Indianapolis Colts':'clt',
         'Arizona Cardinals':'crd','Dallas Cowboys':'dal','Denver Broncos':'den','Detroit Lions':'det',
         'Green Bay Packers':'gnb','Houston Texans':'htx','Jacksonville Jaguars':'jax',
         'Kansas City Chiefs':'kan','Miami Dolphins':'mia','Minnesota Vikings':'min','New Orleans Saints':'nor',
         'New England Patriots':'nwe','New York Giants':'nyg','New York Jets':'nyj','Tennessee Titans':'oti',
         'Philadelphia Eagles':'phi','Pittsburgh Steelers':'pit','Oakland Raiders':'rai',
         'Las Vegas Raiders':'rai','St. Louis Rams':'ram','Los Angeles Rams':'ram','Baltimore Ravens':'rav',
         'San Diego Chargers':'sdg','Los Angeles Chargers':'sdg','Seattle Seahawks':'sea',
         'San Francisco 49ers':'sfo','Tampa Bay Buccaneers':'tam','Washington Redskins':'was'}
cols = ['H_Pts','V_Pts','H_Pts_Opp','V_Pts_Opp','H_Off_Pass','H_O_Pass_Rank','V_Off_Pass','V_O_Pass_Rank','H_Off_Rush',
        'H_O_Rush_Rank','V_Off_Rush','V_O_Rush_Rank','H_Def_Pass','H_D_Pass_Rank','V_Def_Pass','V_D_Pass_Rank',
        'H_Def_Rush','H_D_Rush_Rank','V_Def_Rush','V_D_Rush_Rank','H_TD','V_TD','H_TD_on_Def','V_TD_on_Def',
        'H_FG_Made','V_FG_Made','H_FG_Att','V_FG_Att','H_RZ_Conv','V_RZ_Conv','H_RZ_Att','V_RZ_Att','H_RZ_Def_Conv',
        'V_RZ_Def_Conv','H_RZ_Def_Att','V_RZ_Def_Att','H_Poss','V_Poss','H_Plays','V_Plays','H_TO_Gain','V_TO_Gain',
        'H_TO_Lost','V_TO_Lost','H_Yds_Pen','V_Yds_Pen','H_Sacks_Def','V_Sacks_Def','H_Tackles_Loss','V_Tackles_Loss',
        'H_Kickret','V_Kickret','H_Puntret','V_Puntret']
hdictmap = {'Points':'H_Pts','Points_Opp':'H_Pts_Opp','Yds_Off_Pass':'H_Off_Pass','Yds_Off_Rush':'H_Off_Rush',
           'Yds_Def_Pass':'H_Def_Pass','Yds_Def_Rush':'H_Def_Rush','TD':'H_TD','TD_on_Def':'H_TD_on_Def',
            'FG_Made':'H_FG_Made','FG_Att':'H_FG_Att','RZ_Conv':'H_RZ_Conv','RZ_Att':'H_RZ_Att',
            'RZ_Def_Conv':'H_RZ_Def_Conv','RZ_Def_Att':'H_RZ_Def_Att','Possession':'H_Poss','Plays':'H_Plays',
           'TO_Gained':'H_TO_Gain','TO_Lost':'H_TO_Lost','Yds_Pen':'H_Yds_Pen','Sacks_Def':'H_Sacks_Def',
           'Tackles_Loss':'H_Tackles_Loss','Yds_per_Kickret':'H_Kickret','Yds_per_Puntret':'H_Puntret'}
vdictmap = {'Points':'V_Pts','Points_Opp':'V_Pts_Opp','Yds_Off_Pass':'V_Off_Pass','Yds_Off_Rush':'V_Off_Rush',
           'Yds_Def_Pass':'V_Def_Pass','Yds_Def_Rush':'V_Def_Rush','TD':'V_TD','TD_on_Def':'V_TD_on_Def',
            'FG_Made':'V_FG_Made','FG_Att':'V_FG_Att','RZ_Conv':'V_RZ_Conv','RZ_Att':'V_RZ_Att',
            'RZ_Def_Conv':'V_RZ_Def_Conv','RZ_Def_Att':'V_RZ_Def_Att','Possession':'V_Poss','Plays':'V_Plays',
           'TO_Gained':'V_TO_Gain','TO_Lost':'V_TO_Lost','Yds_Pen':'V_Yds_Pen','Sacks_Def':'V_Sacks_Def',
           'Tackles_Loss':'V_Tackles_Loss','Yds_per_Kickret':'V_Kickret','Yds_per_Puntret':'V_Puntret'}

In [26]:
def weekofgame(game):
    from datetime import timedelta, datetime as dt
    day_of_week={2:0,3:1,4:2,5:3,6:4,0:5,1:6} #day of week starts on Wednesday
    date = dt.strptime(game[0:8],'%Y%m%d')
# .weekday() returns the day of the week for datetime.datetime   
    beg_week = date - timedelta(days=day_of_week[date.weekday()])
    fmt = lambda x: str(x) if x >= 10 else '0'+str(x)
    return(fmt(beg_week.month)+fmt(beg_week.day))

def get_mapping(group):
    d={}
    keys = sorted(group['Week_Beg'].unique())
    values = [i for i in range((num+1),(num+1)+len(keys))]
    for key,value in zip(keys,values):
        d[key] = value
    return d

def get_year(code):
    if (code[4:6] == '01'):
        year = str(int(code[0:4])-1)
    else:
        year = str(int(code[0:4]))
    return year
def normalize_weights(a,n):
    arr = np.exp(a*np.arange(n))
    return n*arr/np.sum(arr)

In [48]:
years = [str(year) for year in range(2010,2020)]
#years = ['2018']

In [224]:
dfgame=pd.DataFrame(columns=cols)
dfgame.index.set_names('Code',inplace=True)

In [231]:
num = 6 # number of games averaged over
wt = normalize_weights(0.5,num)
for year in years:
    path = 'data/df_step2_'+year+'.data'
    with open(path, 'rb') as f:
        dfyear = pickle.load(f)
    for team in dfyear.index.levels[0]:  # Fill in mean values of statistical categories
        dfyear.loc[team].sort_index(inplace=True)
        for row in range(num,len(dfyear.loc[team])):
            code = dfyear.loc[team].index[row]
            if team == code[-3:]:
                for key in hdictmap:
                    dfgame.loc[code,hdictmap[key]] = \
                        (dfyear.loc[team].iloc[row-num:row,dfyear.columns.get_loc(key)]*wt).mean() # Averaged over num games
            else: 
                for key in vdictmap:
                    dfgame.loc[code,vdictmap[key]] = \
                        (dfyear.loc[team].iloc[row-num:row,dfyear.columns.get_loc(key)]*wt).mean()
            
    for row in range(num,16):       # Calculate rank categories
        opass = [(dfyear.loc[team].iloc[row-num:row,dfyear.columns.get_loc('Yds_Off_Pass')]*wt).mean() 
                 for team in dfyear.index.levels[0]]
        oprnk = pd.Series(opass,index=dfyear.index.levels[0]).rank()
        orush = [(dfyear.loc[team].iloc[row-num:row,dfyear.columns.get_loc('Yds_Off_Rush')]*wt).mean() 
                 for team in dfyear.index.levels[0]]
        orrnk = pd.Series(orush,index=dfyear.index.levels[0]).rank()
        dpass = [(dfyear.loc[team].iloc[row-num:row,dfyear.columns.get_loc('Yds_Def_Pass')]*wt).mean() 
                 for team in dfyear.index.levels[0]]
        dprnk = pd.Series(dpass,index=dfyear.index.levels[0]).rank(ascending=False)
        drush = [(dfyear.loc[team].iloc[row-num:row,dfyear.columns.get_loc('Yds_Def_Rush')]*wt).mean() 
                 for team in dfyear.index.levels[0]]
        drrnk = pd.Series(drush,index=dfyear.index.levels[0]).rank(ascending=False)
        for team in dfyear.index.levels[0]:
            if row < len(dfyear.loc[team]): 
                code = dfyear.loc[team].index[row]
                if team == code[-3:]: 
                    dfgame.loc[code,'H_O_Pass_Rank'] = oprnk[team]
                    dfgame.loc[code,'H_O_Rush_Rank'] = orrnk[team]
                    dfgame.loc[code,'H_D_Pass_Rank'] = dprnk[team]
                    dfgame.loc[code,'H_D_Rush_Rank'] = drrnk[team]
                else: 
                    dfgame.loc[code,'V_O_Pass_Rank'] = oprnk[team]
                    dfgame.loc[code,'V_O_Rush_Rank'] = orrnk[team]
                    dfgame.loc[code,'V_D_Pass_Rank'] = dprnk[team]
                    dfgame.loc[code,'V_D_Rush_Rank'] = drrnk[team]

dfgame.dropna(inplace=True)
dfgame[cols] = dfgame[cols].astype('float64')

In [ ]:
ginfo = pd.DataFrame(index=dfgame.index,columns=['Surface','Temperature','Wind','Over/Under'])
codes = ginfo.index
for code in codes:
    url='https://www.pro-football-reference.com/boxscores/'+code+'.htm'
    res=requests.get(url)
    comm = re.compile('<!--|-->')
# Game Info
    table=pd.read_html(comm.sub("",res.text),attrs={'id':'game_info'},flavor='bs4')
    table = table[0]
    table = table.set_index(table.columns[0])
    ginfo.loc[code,'Surface'] = table.loc['Surface',1]
    ginfo.loc[code,'Over/Under'] = float(table.loc['Over/Under',1].split(' ')[0])
    if 'Weather' in table.index:
        string = table.loc['Weather',1]
        ginfo.loc[code,'Temperature'] = int(string.split(' ')[0])
        if 'mph' in string:
            for substr in string.split(','):
                if 'mph' in substr:
                    ginfo.loc[code,'Wind'] = int(''.join(filter(str.isdigit, substr)))
        else:
            ginfo.loc[code,'Wind'] = 0

In [403]:
rh = pd.DataFrame(index=ginfo.index,columns = ['Humidity'])
codes = ginfo.index
for counter, code in enumerate(codes):
    f = open('code','w')
    f.write(str(counter) + ',' + str(code))
    f.close()
    url='https://www.pro-football-reference.com/boxscores/'+code+'.htm'
    res=requests.get(url)
    comm = re.compile('<!--|-->')
# Game Info
    table=pd.read_html(comm.sub("",res.text),attrs={'id':'game_info'},flavor='bs4')
    table = table[0]
    table = table.set_index(table.columns[0])
    if 'Weather' in table.index:
        string = table.loc['Weather',1]
        if 'humidity' in string:
            for substr in string.split(','):
                if 'humidity' in substr:
                    rh.loc[code,'Humidity'] = int(''.join(filter(str.isdigit, substr)))
        else:
            rh.loc[code,'Humidity'] = 0
    else:
        rh.loc[code,'Humidity'] = 0 

In [410]:
rh['Humidity']=pd.to_numeric(rh.Humidity)

In [411]:
ginfo['Humidity'] = rh['Humidity']

In [423]:
path='data/df_gameinfo.pkl'
with open(path, 'wb') as f:
    pickle.dump(ginfo,f)

In [71]:
path='data/df_gameinfo.pkl'
with open(path, 'rb') as f:
    ginfo=pickle.load(f)

In [ ]:
# Actual game temperatures manually entered
tmanu={'201912010jax':83,'201910270jax':84,'201911030sdg':77,'201912010mia':82,
       '201912080jax':68,'201912290jax':74,'201911100gnb':36,'201911100cle':53,
       '201812090sdg':72,'201712100jax':55,'201610230nyj':62,'201512130nyj':66}
for key in tmanu.keys():
    ginfo.loc[key,'Temperature'] = tmanu[key]
# Imputed for dome games
ginfo['Temperature']=ginfo['Temperature'].fillna(70)

In [427]:
len(ginfo),len(ginfo[ginfo.Humidity>0])

(2559, 1306)

In [227]:
dfgame = dfgame.join(ginfo,how='left')

In [212]:
dfgame = pd.concat([dfgame[['Over/Under']],dfgame.drop('Over/Under',axis=1)],axis=1)

In [ ]:
path='data/df_gameinfo.pkl'
with open(path, 'wb') as f:
    pickle.dump(ginfo,f)

In [190]:
path='data/df_analyze1.pkl'
with open(path, 'wb') as f:
    pickle.dump(dfgame,f)

In [384]:
path='data/df_analyze6.pkl'
with open(path, 'rb') as f:
    dfgame = pickle.load(f)

In [214]:
dfgame['H_Pass_Metric'] = (dfgame['H_O_Pass_Rank'] + dfgame['V_D_Pass_Rank'])/2.0
dfgame['H_Rush_Metric'] = (dfgame['H_O_Rush_Rank'] + dfgame['V_D_Rush_Rank'])/2.0
dfgame['V_Pass_Metric'] = (dfgame['V_O_Pass_Rank'] + dfgame['H_D_Pass_Rank'])/2.0
dfgame['V_Rush_Metric'] = (dfgame['V_O_Rush_Rank'] + dfgame['H_D_Rush_Rank'])/2.0
dfrank = pd.concat([dfgame.pop(x) for x in ['H_O_Pass_Rank','H_O_Rush_Rank','V_O_Pass_Rank','V_O_Rush_Rank',
                                            'V_D_Pass_Rank','V_D_Rush_Rank','H_D_Pass_Rank','H_D_Rush_Rank']],
                                            axis=1)

In [215]:
dfgame[dfgame['H_FG_Att']==0]['H_FG_Pct'] = 0.0
ind = dfgame[dfgame['H_FG_Att']!=0].index
dfgame.loc[ind,'H_FG_Pct'] = dfgame.loc[ind,'H_FG_Made']/dfgame.loc[ind,'H_FG_Att']

dfgame[dfgame['V_FG_Att']==0]['V_FG_Pct'] = 0.0
ind = dfgame[dfgame['V_FG_Att']!=0].index
dfgame.loc[ind,'V_FG_Pct'] = dfgame.loc[ind,'V_FG_Made']/dfgame.loc[ind,'V_FG_Att']

dffg = pd.concat([dfgame.pop(x) for x in ['H_FG_Made','H_FG_Att','V_FG_Made','V_FG_Att']],axis=1)

In [216]:
dfgame[dfgame['H_RZ_Att']==0]['H_RZ_Pct'] = 0.0
ind = dfgame[dfgame['H_RZ_Att']!=0].index
dfgame.loc[ind,'H_RZ_Pct'] = dfgame.loc[ind,'H_RZ_Conv']/dfgame.loc[ind,'H_RZ_Att']

dfgame[dfgame['V_RZ_Att']==0]['V_RZ_Pct'] = 0.0
ind = dfgame[dfgame['V_RZ_Att']!=0].index
dfgame.loc[ind,'V_RZ_Pct'] = dfgame.loc[ind,'V_RZ_Conv']/dfgame.loc[ind,'V_RZ_Att']

dfgame[dfgame['H_RZ_Def_Att']==0]['H_RZ_Def_Pct'] = 0.0
ind = dfgame[dfgame['H_RZ_Def_Att']!=0].index
dfgame.loc[ind,'H_Def_RZ_Pct'] = dfgame.loc[ind,'H_RZ_Def_Conv']/dfgame.loc[ind,'H_RZ_Def_Att']

dfgame[dfgame['V_RZ_Def_Att']==0]['V_RZ_Def_Pct'] = 0.0
ind = dfgame[dfgame['V_RZ_Def_Att']!=0].index
dfgame.loc[ind,'V_Def_RZ_Pct'] = dfgame.loc[ind,'V_RZ_Def_Conv']/dfgame.loc[ind,'V_RZ_Def_Att']

dfrz = pd.concat([dfgame.pop(x) for x in ['H_RZ_Conv','H_RZ_Att','V_RZ_Conv','V_RZ_Att']],axis=1)
dfrzdef = pd.concat([dfgame.pop(x) for x in ['H_RZ_Def_Conv','H_RZ_Def_Att','V_RZ_Def_Conv','V_RZ_Def_Att']],
                    axis=1)

In [217]:
dfgame = pd.concat([dfgame.drop('Surface', axis = 1), pd.get_dummies(dfgame.Surface)], axis = 1)

In [64]:
new_gcols=['Home_Team','Vis_Team','H_Game','V_Game','Over/Under','Tot_Pts','H_Pts',
           'H_Pts_Opp','V_Pts','V_Pts_Opp','H_Off_Pass','H_Pass_Metric','V_Off_Pass',
           'V_Pass_Metric','H_Off_Rush','H_Rush_Metric','V_Off_Rush','V_Rush_Metric',
           'H_Def_Pass','V_Def_Pass','H_Def_Rush','V_Def_Rush','H_TD','V_TD',
           'H_TD_on_Def','V_TD_on_Def','H_FG_Pct','V_FG_Pct','H_RZ_Pct','V_RZ_Pct',
           'H_Def_RZ_Pct','V_Def_RZ_Pct','H_Poss','V_Poss','H_Plays','V_Plays',
           'H_TO_Gain','V_TO_Gain','H_TO_Lost','V_TO_Lost','H_Yds_Pen','V_Yds_Pen',
           'H_Sacks_Def','V_Sacks_Def','H_Tackles_Loss','V_Tackles_Loss','H_Kickret',
           'V_Kickret','H_Puntret','V_Puntret','Temperature','Wind','Humidity','a_turf','astroplay',
           'astroturf','fieldturf','grass','matrixturf','sportturf']

In [218]:
years = [str(year) for year in range(2010,2020)]
codes = dfgame.index
for year in years:
    path = 'data/df_step2_'+year+'.data'
    with open(path, 'rb') as f:
        dfyear = pickle.load(f)
    codesubs = [code for code in codes 
      if (((code[0:4] == year) & (code[4:6] != '01')) | (code[0:6] == (str(int(year)+1)+'01')))]
    for code in codesubs:
        home = code[-3:]
        dfgame.loc[code,'Vis_Team'] = dfyear.loc[(home,code),'Opponent']
        dfgame.loc[code,'V_Game'] = dfyear.loc[(home,code),'Points_Opp']
        dfgame.loc[code,'Home_Team'] = \
            dfyear.loc[(teams[dfgame.loc[code,'Vis_Team']],code),'Opponent']
        dfgame.loc[code,'H_Game'] = dfyear.loc[(home,code),'Points']
        dfgame.loc[code,'Tot_Pts'] = dfyear.loc[(home,code),'Points'] + dfyear.loc[(home,code),'Points_Opp']

In [219]:
dfgame=dfgame[new_gcols]

In [220]:
by_week = dfgame[['Tot_Pts','Temperature']]
for ind in by_week.index: 
    by_week.loc[ind,'Week_Beg'] = weekofgame(ind)
    by_week.loc[ind,'Year'] = get_year(ind)
grouped_by_year = by_week.groupby(by_week['Year'])
for key, item in grouped_by_year:
    mapping = get_mapping(item)
    for code in item.index:
        by_week.loc[code,'Week'] = int(mapping[by_week.loc[code,'Week_Beg']])
by_week['Week'] = by_week['Week'].astype('int')
dfgame['Week'] = by_week['Week']

# Wind chill factor for temperature when T <= 50 °F and wind > 3 mph
ind = dfgame[(dfgame['Temperature'] <= 50) & (dfgame['Wind'] > 3)].index
dfgame.loc[ind,'Temperature'] = (35.74 + 0.6215*dfgame.loc[ind,'Temperature'] - 
                                 35.75*(dfgame.loc[ind,'Wind']**0.16) + 
                                 0.4275*dfgame.loc[ind,'Temperature']*(dfgame.loc[ind,'Wind']**0.16))

dfgame['dome'] = ((dfgame['Temperature'] == 70) & (dfgame['Wind'] == 0))*1
dfgame.drop(['H_TD','V_TD','astroturf'],axis=1,inplace=True)

In [221]:
# Wind chill factor for temperature when T <= 50 °F and wind > 3 mph
ind = dfgame[(dfgame['Temperature'] <= 50) & (dfgame['Wind'] > 3)].index
dfgame.loc[ind,'Temperature'] = (35.74 + 0.6215*dfgame.loc[ind,'Temperature'] - 
                                35.75*(dfgame.loc[ind,'Wind']**0.16) + 
                                0.4275*dfgame.loc[ind,'Temperature']*(dfgame.loc[ind,'Wind']**0.16))

In [199]:
dfgame

,Home_Team,Vis_Team,H_Game,V_Game,Over/Under,Tot_Pts,H_Pts,H_Pts_Opp,V_Pts,V_Pts_Opp,...,Wind,Humidity,a_turf,astroplay,fieldturf,grass,matrixturf,sportturf,Week,dome
Code,,,,,,,,,,,,,,,,,,,,,
201009190atl,Atlanta Falcons,Arizona Cardinals,41.0,7.0,43.5,48.0,9.0,15.0,17.0,13.0,...,0.0,0,0,0,1,0,0,0,2,1
201009260nor,New Orleans Saints,Atlanta Falcons,24.0,27.0,49.5,51.0,25.0,22.0,41.0,7.0,...,0.0,0,0,0,0,0,0,1,3,1
201010030atl,Atlanta Falcons,San Francisco 49ers,16.0,14.0,42.5,30.0,27.0,24.0,10.0,31.0,...,0.0,0,0,0,1,0,0,0,4,1
201010100cle,Cleveland Browns,Atlanta Falcons,10.0,20.0,41.5,30.0,23.0,20.0,16.0,14.0,...,9.0,0,0,0,0,1,0,0,5,0
201010170phi,Philadelphia Eagles,Atlanta Falcons,31.0,17.0,43.0,48.0,27.0,24.0,20.0,10.0,...,12.0,45,0,0,0,1,0,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912010rav,Baltimore Ravens,San Francisco 49ers,20.0,17.0,45.0,37.0,45.0,6.0,37.0,8.0,...,13.0,91,0,0,0,1,0,0,13,0
201911030sea,Seattle Seahawks,Tampa Bay Buccaneers,40.0,34.0,51.0,74.0,27.0,20.0,23.0,27.0,...,6.0,66,0,0,1,0,0,0,9,0
201911110sfo,San Francisco 49ers,Seattle Seahawks,24.0,27.0,47.0,51.0,28.0,25.0,40.0,34.0,...,3.0,32,0,0,0,1,0,0,10,0


In [222]:
path = 'data/df_working6.pkl'
with open(path, 'wb') as f:
    pickle.dump(dfgame,f)

In [218]:
dfyear

Home/Away             Opponent  Points  Points_Opp  \
Team Code                                                              
atl  201909080min         A    Minnesota Vikings      12          28   
     201909150atl         H  Philadelphia Eagles      24          20   
     201909220clt         A   Indianapolis Colts      24          27   
     201909290atl         H     Tennessee Titans      10          24   
     201910060htx         A       Houston Texans      32          53   
...                     ...                  ...     ...         ...   
was  201912010car         A    Carolina Panthers      29          21   
     201912080gnb         A    Green Bay Packers      15          20   
     201912150was         H  Philadelphia Eagles      27          37   
     201912220was         H      New York Giants      35          41   
     201912290dal         A       Dallas Cowboys      16          47   

                   Yds_Off_Pass  Yds_Off_Rush  Yds_Def_Pass  Yds_Def_Rush  TD  \
Team Code                                                                       
atl  201909080min           304            73            98           172   2   
     201909150atl           320            57           255            49   3   
     201909220clt           304            93           310            79   3   
     201909290atl           397            58           227           138   1   
     201910060htx           330            57           426           166   4   
...                         ...           ...           ...           ...  ..   
was  201912010car           147           248           278            65   3   
     201912080gnb           170           121           195           174   2   
     201912150was           261           101           266           157   3   
     201912220was           302            80           352           206   5   
     201912290dal           206            88           303           223   1   

                   TD_on_Def  ...  RZ_Def_Att  Possession  Plays  TO_Gained  \
Team Code                     ...                                             
atl  201909080min          0  ...           4   30.216667     67          0   
     201909150atl          0  ...           5   27.100000     63          3   
     201909220clt          0  ...           6   28.616667     55          0   
     201909290atl          0  ...           4   30.683333     77          0   
     201910060htx          0  ...           7   29.000000     69          1   
...                      ...  ...         ...         ...    ...        ...   
was  201912010car          0  ...           4   30.616667     61          2   
     201912080gnb          0  ...           5   29.600000     60          1   
     201912150was          0  ...           5   23.050000     53          1   
     201912220was          0  ...           3   33.800000     64          0   
     201912290dal          0  ...           3   25.750000     62          1   

                   TO_Lost  Yds_Pen  Sacks_Def  Tackles_Loss  Yds_per_Kickret  \
Team Code                                                                       
atl  201909080min        3       78        1.0             4        22.000000   
     201909150atl        3       58        3.0             4        25.000000   
     201909220clt        1      128        1.0             5        22.500000   
     201909290atl        1       50        0.0             6         0.000000   
     201910060htx        1       63        0.0             3        16.000000   
...                    ...      ...        ...           ...              ...   
was  201912010car        0       65        7.0             5        16.000000   
     201912080gnb        1       25        4.0             6         0.000000   
     201912150was        1       60        2.0             3        29.666667   
     201912220was        0       51        1.0             6        25.000000   
     201912290dal        2 